<table style="border: none" align="left">
   <tr style="border: none">
      <th style="border: none"><font face="verdana" size="5" color="black"><b>Predicting customer churn with IBM Watson Machine Learning</b></th>
      <th style="border: none"><img src="https://github.com/pmservice/customer-satisfaction-prediction/blob/master/app/static/images/ml_icon_gray.png?raw=true" alt="Watson Machine Learning icon" height="40" width="40"></th>
  <tr style="border: none">
       <th style="border: none"><img src="https://github.com/pmservice/wml-sample-models/blob/master/spark/customer-satisfaction-prediction/images/users_banner_2-03.png?raw=true" width="600" alt="Icon"> </th>
   </tr>
</table>

This notebook contains steps to deploy sample SPSS stream, and start scoring new data. 

Some familiarity with bash is helpful. This notebook uses Python 3.

You will use a data set, **Telco Customer Churn**, which details anonymous customers data of telecommunication company. Use the details of this data set to predict customer churn which is very critical to business as it's easier to retain existing customers rather than acquiring new ones.

## Learning goals

The learning goals of this notebook are:

-  Working with the instance
-  Online deployment of SPSS model
-  Scoring data using deployed model


## Contents

This notebook contains the following parts:

1.	[Setup](#setup)
2.	[Work with service instance](#instance) 
3.	[Persist, deploy and score in a Cloud](#scoring)
4.	[Summary and next steps](#summary)

<a id="setup"></a>
## 1. Setup

Before you use the sample code in this notebook, you must perform the following setup tasks:

-  Create a [Watson Machine Learning Service](https://console.ng.bluemix.net/catalog/services/ibm-watson-machine-learning/) instance required for model storing and deployment (a free plan is offered).
-   Install [watson-machine-learning-client](https://pypi.python.org/pypi?%3Aaction=search&term=watson-machine-learning-client&submit=search) (not required on DSx).

Install python API client from pypi.

In [1]:
!rm -rf $PIP_BUILD/watson-machine-learning-client

In [2]:
!pip install watson-machine-learning-client --upgrade

Requirement already up-to-date: watson-machine-learning-client in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages
Requirement not upgraded as not directly required: urllib3 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from watson-machine-learning-client)
Requirement not upgraded as not directly required: tqdm in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from watson-machine-learning-client)
Requirement not upgraded as not directly required: certifi in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from watson-machine-learning-client)
Requirement not upgraded as not directly required: tabulate in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from watson-machine-learning-client)
Requirement not upgraded as not directly required: pandas in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from watson-machine-learning-client)
Requirement not upgraded as not directly required: lomond in /opt/conda/envs/DSX-Python35/lib/p

<a id="instance"></a>
## 2. Working with the instance

In this section you will learn how to get details of your instance.

**Action**: Put credentials from your instance of Watson Machine Learning service in IBM Cloud here.

In [3]:
wml_credentials = {
  "url": "https://ibm-watson-ml.mybluemix.net",
  "access_key": "***",
  "username": "***",
  "password": "***",
  "instance_id": "***"
}

In [4]:
# The code was removed by DSX for sharing.

In [5]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient

In [6]:
client = WatsonMachineLearningAPIClient(wml_credentials)

Details about your Watson Machine Learning service are available in instance_details.

In [8]:
instance_details = client.instance_details

Getting stored models on WML instance

In [9]:
client.repository.list_models()

------------------------------------  ----------------------------------  ------------------------  --------------
GUID                                  NAME                                CREATED                   FRAMEWORK
43986a55-0f0e-493b-9b98-0712f918b0ba  MNIST CNN model                     2018-03-15T16:12:06.503Z  tensorflow-1.5
a5a952e1-fa6f-4c64-88f7-293b367b7e1f  MNIST best model                    2018-03-16T10:12:16.773Z  tensorflow-1.5
c755c740-dab6-4d5d-b873-1060885d55d5  MNIST best model                    2018-03-16T10:12:31.990Z  tensorflow-1.5
861b6b73-6861-49c0-867c-d171d6d9d545  MNIST best model                    2018-03-16T13:00:40.461Z  tensorflow-1.5
f7219c8f-fccb-4d26-a1bd-f51544090eae  training-S3ttnlgig_2 best model     2018-03-16T13:32:33.747Z  tensorflow-1.5
327242a2-d2c0-400f-b673-4f33ad7ec17d  product-line-prediction             2018-03-16T18:08:31.260Z  mllib-2.0
a0cb79ca-e937-4cbf-b7e5-23dad2d2ecec  drug-selection                      2018-03-16T18:10

Geting list of deployed models.

In [10]:
client.deployments.list()

------------------------------------  -------------------------------  ------  --------------  ------------------------  --------------
GUID                                  NAME                             TYPE    STATE           CREATED                   FRAMEWORK
0efa6832-80ea-4637-baed-771827562112  MNIST keras deployment           online  DEPLOY_FAILURE  2018-03-15T16:12:48.062Z  tensorflow-1.5
1bf606e8-e8c4-4143-8727-5b2d1ac97b7e  MNIST keras deployment           online  DEPLOY_SUCCESS  2018-03-16T13:00:52.773Z  tensorflow-1.5
2436e614-5396-4b78-ac02-3e6a69479b1c  MNIST keras deployment           online  DEPLOY_FAILURE  2018-03-15T16:13:29.910Z  tensorflow-1.5
4bcae2dc-b309-4e0d-8858-2c04fe4c6383  Mnist model deployment           online  DEPLOY_SUCCESS  2018-03-28T09:02:49.220Z  tensorflow-1.5
4fb26af0-e4b5-4fcc-9156-2fe6dcad41a2  drug deployment                  online  DEPLOY_SUCCESS  2018-03-16T18:11:33.498Z  mllib-2.0
687c8e42-86e8-4b2c-a1bb-c9f2ebebfa23  MNIST keras deployme

<a id="scoring"></a>
## 3. Persist, deploy and score model in a Cloud

In this section you will learn how to create online deployment and to score a new data record by using the Watson Machine Learning REST API. 
For more information about REST APIs, see the [Swagger Documentation](http://watson-ml-api.mybluemix.net/).

### 3.1: Persist model

Download SPSS sample model from git project.

**Note:** You may need to install first wget package by running following code: `!pip install wget`

In [12]:
!pip install wget

  Running setup.py bdist_wheel for wget ... done
  Stored in directory: /home/dsxuser/.cache/pip/wheels/6d/98/29/61ccc41148f871009126c2e844e26f73eeb25e12cca92228a5
Successfully built wget


In [13]:
import os, wget

sample_dir = 'spss_sample_model'
if not os.path.isdir(sample_dir):
    os.mkdir(sample_dir)

filename=os.path.join(sample_dir, 'customer-satisfaction-prediction.str')
if not os.path.isfile(filename):
    filename = wget.download('https://github.com/pmservice/wml-sample-models/raw/master/spss/customer-satisfaction-prediction/model/customer-satisfaction-prediction.str',\
                             out=sample_dir)
print(filename)

spss_sample_model/customer-satisfaction-prediction.str


Store SPSS sample model in your Watson Machine Learning instance.

In [21]:
props_spss = {client.repository.ModelMetaNames.NAME: 'SPSS model for Churn prediction',
              client.repository.ModelMetaNames.AUTHOR_NAME: "IBM", 
              client.repository.ModelMetaNames.AUTHOR_EMAIL: "ibm@ibm.com", 
              client.repository.ModelMetaNames.FRAMEWORK_NAME: "spss-modeler", 
              client.repository.ModelMetaNames.FRAMEWORK_VERSION: "18.1"}

model_details = client.repository.store_model(filename, props_spss)

Geting details of deployed models.

In [22]:
print(model_details)

{'metadata': {'created_at': '2018-03-28T13:20:40.791Z', 'url': 'https://ibm-watson-ml.mybluemix.net/v3/wml_instances/3f6e5c2b-4880-46aa-9d79-62e90ccc9d56/published_models/9fb96bc1-bc12-4717-a03f-9b009a32d578', 'modified_at': '2018-03-28T13:20:40.855Z', 'guid': '9fb96bc1-bc12-4717-a03f-9b009a32d578'}, 'entity': {'deployments': {'count': 0, 'url': 'https://ibm-watson-ml.mybluemix.net/v3/wml_instances/3f6e5c2b-4880-46aa-9d79-62e90ccc9d56/published_models/9fb96bc1-bc12-4717-a03f-9b009a32d578/deployments'}, 'author': {'email': 'ibm@ibm.com', 'name': 'IBM'}, 'runtime_environment': 'None Provided', 'model_type': 'spss-modeler-18.1', 'feedback_url': 'https://ibm-watson-ml.mybluemix.net/v3/wml_instances/3f6e5c2b-4880-46aa-9d79-62e90ccc9d56/published_models/9fb96bc1-bc12-4717-a03f-9b009a32d578/feedback', 'learning_iterations_url': 'https://ibm-watson-ml.mybluemix.net/v3/wml_instances/3f6e5c2b-4880-46aa-9d79-62e90ccc9d56/published_models/9fb96bc1-bc12-4717-a03f-9b009a32d578/learning_iterations', 

### 3.1: Create online deployment

Using below commands you can create online deployment.

In [23]:
model_uid = client.repository.get_model_uid(model_details)
model_deployment = client.deployments.create(model_uid, "Sample SPSS model deployment")



#######################################################################################

Synchronous deployment creation for uid: '3142ff00-abbb-45c2-83c2-63a3e6d5b8bd' started

#######################################################################################


INITIALIZING
DEPLOY_IN_PROGRESS......
DEPLOY_SUCCESS


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='3142ff00-abbb-45c2-83c2-63a3e6d5b8bd'
------------------------------------------------------------------------------------------------




Now you can check your deployments details.

In [18]:
deployment_uid = client.deployments.get_uid(model_deployment)
print(client.deployments.get_details(deployment_uid))

{'metadata': {'created_at': '2018-03-28T13:11:41.657Z', 'url': 'https://ibm-watson-ml.mybluemix.net/v3/wml_instances/3f6e5c2b-4880-46aa-9d79-62e90ccc9d56/published_models/5bcf67ab-cd88-49c9-99fd-99dde60eff70/deployments/0208ad3b-a869-49ff-a15f-51667143aed8', 'modified_at': '2018-03-28T13:11:41.921Z', 'guid': '0208ad3b-a869-49ff-a15f-51667143aed8'}, 'entity': {'published_model': {'author': {'email': 'ibm@ibm.com', 'name': 'IBM'}, 'url': 'https://ibm-watson-ml.mybluemix.net/v3/wml_instances/3f6e5c2b-4880-46aa-9d79-62e90ccc9d56/published_models/5bcf67ab-cd88-49c9-99fd-99dde60eff70', 'guid': '5bcf67ab-cd88-49c9-99fd-99dde60eff70', 'created_at': '2018-03-28T13:11:41.633Z', 'name': 'SPSS model for Churn prediction'}, 'scoring_url': 'https://ibm-watson-ml.mybluemix.net/v3/wml_instances/3f6e5c2b-4880-46aa-9d79-62e90ccc9d56/published_models/5bcf67ab-cd88-49c9-99fd-99dde60eff70/deployments/0208ad3b-a869-49ff-a15f-51667143aed8/online', 'status_details': {'status': 'DEPLOY_SUCCESS'}, 'status': 'DE

### 3.2: Scoring

You can get `scoring_url` for your deployed model using commands below.

In [19]:
scoring_url = client.deployments.get_scoring_url(model_deployment)
print(scoring_url)

https://ibm-watson-ml.mybluemix.net/v3/wml_instances/3f6e5c2b-4880-46aa-9d79-62e90ccc9d56/published_models/5bcf67ab-cd88-49c9-99fd-99dde60eff70/deployments/0208ad3b-a869-49ff-a15f-51667143aed8/online


You can score data using client `score` method.

In [20]:
scoring_payload = {"fields":["customerID","gender","SeniorCitizen","Partner","Dependents","tenure","PhoneService","MultipleLines","InternetService","OnlineSecurity","OnlineBackup","DeviceProtection","TechSupport","StreamingTV","StreamingMovies","Contract","PaperlessBilling","PaymentMethod","MonthlyCharges","TotalCharges","Churn","SampleWeight"],"values":[["3638-WEABW","Female",0,"Yes","No",58,"Yes","Yes","DSL","No","Yes","No","Yes","No","No","Two year","Yes","Credit card (automatic)",59.9,3505.1,"No",2.768]]}
prediction = client.deployments.score(scoring_url, scoring_payload)

print(prediction)

{'fields': ['customerID', 'Churn', 'Predicted Churn', 'Probability of Churn'], 'values': [['3638-WEABW', 'No', 'No', 0.0526309571556145]]}


As we can see this sample telco customer is satisfied ("Predicted Churn", "No").

<a id="summary"></a>
## 7. Summary and next steps     

 You successfully completed this notebook! You learned how to use Watson Machine Learning for SPSS model deployment and scoring. Check out our _[Online Documentation](https://console.ng.bluemix.net/docs/services/PredictiveModeling/index.html?pos=2)_ for more samples, tutorials, documentation, how-tos, and blog posts. 

### Authors

Lukasz Cmielowski, PhD, is a Automation Architect and Data Scientist in IBM with a track record of developing enterprise-level applications that substantially increases clients' ability to turn data into actionable knowledge.

Copyright © 2017 IBM. This notebook and its source code are released under the terms of the MIT License.